In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib qt

In [94]:
# HP parameters
def kernel(t, alpha=0.4, beta=3):
    return alpha * t * np.exp(-beta*t)

mu = 0.5

In [95]:
timevect = np.linspace(0, 10, 100)
plt.figure()
plt.plot(timevect, kernel(timevect))
plt.grid()

In [96]:
def intensity(t, past_arrivals, mu, h):
    """
    t: time at which to evaluate lambda*(t)
    past_arrivals: array containing past arrival times
    mu: background intensity
    h: excitation function

    computes lambda*(t) = mu + sum h(t-ti) for ti<t
    """
    return mu + sum(h(t - arrival) for arrival in past_arrivals if arrival < t)

### Generate a univariate HP by thinning

In [ ]:
# Thinning algorithm to generate arrivals
eps = 10e-10
P = np.array([])  # array of arrivals
T = 40  # generate process on [0, T]
t = 0

while t < T:
    M = intensity(t=t+eps, past_arrivals=P, mu=mu, h=kernel)
    E = np.random.exponential(1/M)
    t = t + E
    U = np.random.uniform(low=0, high=M)

    if t < T and U <= intensity(t=t, past_arrivals=P, mu=mu, h=kernel):
        P = np.hstack([P, t])

In [98]:
# Plot counting process
def counting_process(arrival_times):
    return [i + 1 for i in range(len(arrival_times))]

plt.figure()
plt.step(x=P, y=counting_process(P), where='post')
plt.grid()
plt.xlabel('Time')
plt.ylabel('Count')
plt.scatter(P, [0] * len(P), marker='x', color='red', label='Time Values')

In [71]:
# Plot intensity function


In [99]:
# Plot histogram of arrivals